In [15]:
import glob
import tensorflow as tf
import numpy as np

# 이미지 파일들의 경로
file_paths = glob.glob('./stones_png/*/*.png')  # 실제 이미지 파일이 있는 폴더의 경로로 변경해야 합니다.

# 파일 경로를 이미지 데이터셋으로 변환
image_dataset = tf.data.Dataset.from_tensor_slices(file_paths)

# 이미지 파일을 읽어들이고 전처리하는 함수 정의
def preprocess_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=3)  # JPEG 이미지일 경우
    image = tf.image.resize(image, (224, 224))  # 이미지 크기 조정
    image = tf.cast(image, tf.float32) / 255.0  # 이미지 정규화 및 타입 변환
    return image

# 파일 경로 데이터셋을 이미지 데이터셋으로 변환하고 전처리 적용
image_dataset = image_dataset.map(preprocess_image)


In [26]:
import tensorflow as tf

for image in image_dataset.take(1):
    input_shape = image.shape  # 가져온 이미지의 형태 확인
    break

# 이미지 입력을 받도록 변경된 모델

# 이미지의 형태 (세로, 가로, 채널)
# input_shape = (224, 224, 3)
X_input = tf.keras.layers.Input(input_shape=(224,224,3))  # 이미지 입력 레이어

# 이미지 처리에 특화된 Convolutional 레이어 추가
conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(X_input)
maxpool1 = tf.keras.layers.MaxPooling2D((2, 2))(conv1)
conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(maxpool1)
maxpool2 = tf.keras.layers.MaxPooling2D((2, 2))(conv2)
flatten = tf.keras.layers.Flatten()(maxpool2)

# 이후의 Dense 레이어 추가
H = tf.keras.layers.Dense(84, activation='swish')(flatten)
Y = tf.keras.layers.Dense(2, activation='softmax')(H)

# 새로운 모델 정의
model = tf.keras.models.Model(X_input, Y)

# 모델 컴파일
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [27]:
# 모델 학습
model.fit(image_dataset, epochs=10)


Epoch 1/10


ValueError: in user code:

    File "/Users/ddoram/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/Users/ddoram/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/ddoram/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/Users/ddoram/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "/Users/ddoram/anaconda3/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/ddoram/anaconda3/lib/python3.11/site-packages/keras/src/engine/input_spec.py", line 29